In [7]:
import pandas as pd
import sys
import os
from tqdm.auto import tqdm
tqdm.pandas()
sys.path.append('/home/platinsky/Documents/hack/ForecastingExtremeWeather/lib/')
import target_markup
import prepare_dataset
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
weather_data = pd.read_csv('../data/weather_dataset.csv', sep=';')
weather_data.head()

/tmp/ipykernel_127964/109065263.py:1: DtypeWarning: Columns (22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_data = pd.read_csv('../data/weather_dataset.csv', sep=';')


,Unnamed: 0,local_time,T,Po,P,Pa,U,DD,Ff,ff10,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,station_id
0,0,2022-12-09 21:00:00,-7.5,765.6,766.3,1.2,91.0,"Ветер, дующий с востока",1.0,NaN,...,NaN,8.0,-8.8,NaN,NaN,NaN,NaN,NaN,NaN,22550
1,1,2022-12-09 18:00:00,-7.8,764.4,765.2,0.8,90.0,"Ветер, дующий с востоко-юго-востока",2.0,NaN,...,"Перистых, перисто-кучевых или перисто-слоистых...",8.0,-9.2,0.6,12.0,NaN,NaN,NaN,NaN,22550
2,2,2022-12-09 15:00:00,-8.1,763.6,764.5,0.7,89.0,"Ветер, дующий с востока",2.0,NaN,...,"Перистых, перисто-кучевых или перисто-слоистых...",4.0,-9.6,NaN,NaN,NaN,NaN,NaN,NaN,22550
3,3,2022-12-09 12:00:00,-8.3,762.9,763.7,0.8,90.0,"Ветер, дующий с востоко-юго-востока",2.0,NaN,...,"Перистых, перисто-кучевых или перисто-слоистых...",4.0,-9.7,NaN,NaN,NaN,NaN,NaN,NaN,22550
4,4,2022-12-09 09:00:00,-8.8,762.1,762.8,0.7,90.0,"Ветер, дующий с востоко-юго-востока",2.0,NaN,...,NaN,17.0,-10.2,NaN,NaN,NaN,NaN,NaN,NaN,22550


In [3]:
target = pd.json_normalize(weather_data.progress_apply(target_markup.classify_based_on_text, axis=1))
target

  0%|          | 0/365169 [00:00<?, ?it/s]

,VETER,SHKVAL,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
0,False,False,False,True,True,False,False,False
1,False,False,False,True,True,False,False,False
2,False,False,False,True,True,False,False,False
3,False,False,False,True,True,False,False,False
4,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...
365164,False,False,False,True,False,False,False,False
365165,False,False,False,True,True,False,False,False
365166,False,False,False,True,True,False,False,False
365167,False,False,False,True,True,False,False,False


In [4]:
X, Y = prepare_dataset.prepare_dataset(weather_data)

2022-12-11 18:16:03.974 | INFO     | prepare_dataset:prepare_dataset:84 - Current mode of preparation is processed.


  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

2022-12-11 18:16:41.779 | INFO     | prepare_dataset:prepare_dataset:99 - Step: local_time_pipeline
2022-12-11 18:16:41.920 | INFO     | prepare_dataset:prepare_dataset:99 - Step: dd_pipeline


  0%|          | 0/365169 [00:00<?, ?it/s]

...

2022-12-11 18:16:53.334 | INFO     | prepare_dataset:prepare_dataset:99 - Step: h_pipeline


  0%|          | 0/365169 [00:00<?, ?it/s]

2022-12-11 18:16:54.066 | INFO     | prepare_dataset:prepare_dataset:99 - Step: n_pipeline


  0%|          | 0/365169 [00:00<?, ?it/s]

2022-12-11 18:16:54.961 | INFO     | prepare_dataset:prepare_dataset:102 - Filling nans!
2022-12-11 18:17:14.406 | INFO     | prepare_dataset:prepare_dataset:105 - Done (processed)!


In [5]:
target.sum()

VETER          6
SHKVAL         2
METEL       1691
DOZD       79155
SNEG       57246
GRAD        1213
TUMAN       8536
GOLOLED     1193
dtype: int64

In [72]:
from sklearn.linear_model import Ridge,Lasso, LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
for target_column in target.columns:
    y_feat = target[target_column]

    n_true_feat = np.sum(y_feat)
    rand_inds = np.random.randint(0, X.shape[0], size=(n_true_feat * 4))
    true_inds = np.argwhere(y_feat.to_numpy()).flatten()

    all_inds = np.concatenate((rand_inds, true_inds))
    np.random.shuffle(all_inds)

    X_tar = X.iloc[all_inds, :]
    y_tar = y_feat.iloc[all_inds]

    rr = GradientBoostingClassifier()
    X_tar = X_tar.loc[:,X_tar.dtypes[X_tar.dtypes != 'O'].index]
    X_tar = X_tar.drop(columns=['local_time', 'Unnamed: 0'])
    X_tar = X_tar.fillna(0, axis=1)
    rr.fit(X_tar, y_tar)
    X_tested = X.loc[:,X.dtypes[X.dtypes != 'O'].index]
    X_tested = X_tested.drop(columns=['local_time', 'Unnamed: 0'])
    X_tested = X_tested.fillna(0, axis=1)
    print(np.sum(rr.predict(X_tested)))
    print(target_column, sorted(list(zip(np.abs(list(rr.feature_importances_)), X_tar.columns)), reverse=True))

[0. 0. 0. ... 1. 1. 1.]
VETER [(0.2763595034100177, 'Po'), (0.20292207792207784, 'station_id'), (0.18696718146237654, 'Tn_val'), (0.13865074851699252, 'P'), (0.09874710425190925, 'Td'), (0.09635338443662621, 'Pa'), (0.0, 'tR'), (0.0, 'ff3'), (0.0, 'ff10'), (0.0, 'dd_y_rad'), (0.0, 'dd_x_rad'), (0.0, 'dd_isnan'), (0.0, 'dd_changed'), (0.0, 'U'), (0.0, 'Tx_val'), (0.0, 'Tx_isnan'), (0.0, 'Tn_isnan'), (0.0, 'Tg'), (0.0, 'T'), (0.0, 'N'), (0.0, 'H'), (0.0, 'Ff')]
[0. 0. 0. ... 0. 0. 0.]
SHKVAL [(0.6614966542735731, 'T'), (0.11031503134097813, 'P'), (0.08133641194222847, 'U'), (0.06660516780501878, 'station_id'), (0.05330540669132547, 'tR'), (0.024725052440818514, 'Pa'), (0.0021491209563154084, 'Po'), (6.71545497421656e-05, 'Tx_val'), (0.0, 'ff3'), (0.0, 'ff10'), (0.0, 'dd_y_rad'), (0.0, 'dd_x_rad'), (0.0, 'dd_isnan'), (0.0, 'dd_changed'), (0.0, 'Tx_isnan'), (0.0, 'Tn_val'), (0.0, 'Tn_isnan'), (0.0, 'Tg'), (0.0, 'Td'), (0.0, 'N'), (0.0, 'H'), (0.0, 'Ff')]
